# CADS API Python client Tests

In [1]:
import os
import xarray as xr

import cads_api_client

In [2]:
api_url = os.getenv("CADS_API_ROOT_URL", "http://cds2-dev.bopen.eu/")
api_url = os.getenv("CADS_API_ROOT_URL", "http://localhost:8080/api")
api_url

'http://localhost:8080/api'

## Client instantiation

The client expose the APIs for both 
- catalogue exploration 
- data retrieve

In [3]:
client = cads_api_client.ApiClient(api_url)
client

ApiClient()

## 1. Catalogue Exploration


### 1.1 Collections

**Objective**: Verify the access to the list of **collections** and its description

_**expected result**: correct instatiation of collections object._
*if `collections.response` has 200 status code, the request is succesfull and the test can proceed*

In [4]:
collections = client.collections()
collections.response

<Response [200]>

_**expected result**: list of all available collections._

In [5]:
collections.collection_ids()

['cams-global-reanalysis-eac4-monthly',
 'reanalysis-era5-single-levels',
 'reanalysis-era5-land',
 'reanalysis-era5-pressure-levels',
 'reanalysis-era5-land-monthly-means',
 'derived-near-surface-meteorological-variables']

### 1.2 Collection

**Objective**: Verify the access to the list of **collection** and it description

_**expected result**: correct instatiation of collection object._

*If collection.response has 200 status code, the request is succesfull and the test can proceed*

In [6]:
collection = client.collection("reanalysis-era5-pressure-levels")
collection

Collection(response=<Response [200]>, headers={'PRIVATE-TOKEN': 'mysecretpat'})

_**expected result**: json of the collection response describing the collection, containing the keys:_
- _id_
- _title_
- _description_

In [7]:
collection.json

{'type': 'Collection',
 'id': 'reanalysis-era5-pressure-levels',
 'stac_version': '1.0.0',
 'title': 'ERA5 hourly data on pressure levels from 1959 to present',
 'description': '**ERA5** is the fifth generation ECMWF reanalysis for the global climate and weather for the past 4 to 7 decades.\nCurrently data is available from 1950, with Climate Data Store entries for 1950-1978 (preliminary back extension) and from 1959 onwards (final release plus timely updates, this page).\nERA5 replaces the ERA-Interim reanalysis.\n\nReanalysis combines model data with observations from across the world into a globally complete and consistent dataset using the laws of physics. This principle, called data assimilation, is based on the method used by numerical weather prediction centres, where every so many hours (12 hours at ECMWF) a previous forecast is combined with newly available observations in an optimal way to produce a new best estimate of the state of the atmosphere, called analysis, from which

## 3 Data Retrieval: adavanced API


### 3.1 Request Submission: big data request

**Objective:** Verify the capability of the client to submit a request 



**client.submit** function is non blocking function and returns a remote object that allows the monitoring of the request status


_**expected result**: the client submit the requests and return a remote object that allows the process monitoring._

collection = client.collection("reanalysis-era5-pressure-levels")
remote = collection.submit( 
    product_type="reanalysis", 
    variable="temperature", 
    pressure_level="1", 
    year="1971", 
    month=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    day=[
        '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',
        '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
        '25', '26', '27', '28', '29', '30', '31',
    ],
    time="06:00",
    target="test02.grib",
)
remote.request_uid

In [8]:
collection = client.collection("derived-near-surface-meteorological-variables")
remote = collection.submit( 
    month =  ["01"], 
    reference_dataset = ["cru"], 
    variable = ["surface_downwelling_longwave_radiation", "surface_air_pressure"], 
    version = ["1.1"], 
    year = ["1982"]
)
remote.request_uid

HTTPError: 500 Server Error: Internal Server Error for url: http://localhost:8080/api/retrieve/v1/processes/derived-near-surface-meteorological-variables/execute

### 3.2 Request Monitoring

**Objectives**: Verify client capability monitor the requesst

**remote.status** allows to monitor the process status

_**expected result**: returns the updated status of the request: failed, successful, running_

In [ ]:
remote.status

**client.get_requests** returns the list of requests submitted 

_**expected result**: ids of the submitted requests._

In [ ]:
requests = client.get_requests()
requests.job_ids()

_**expected result**: the remote.request_uid is in the list of the submitted requests._

In [ ]:
remote.request_uid in requests.job_ids()

### 3.3 Data download

**Objectives**: Verify client capability to download the data

**download** function is blocking: 
- waits until the requests is completed
- downloads the data

"_**expected result**: the data is downloaded and saved in $output_path._"

In [ ]:
output_path = remote.download("test02.grib")
output_path

In [ ]:
ls -l $ouput_path

_**expected output**: data compatible with the request:_
- _one variable: t (temperature)_
- _three dimensions time (365), latitude (721) and longitude (1440)_

In [ ]:
ds = xr.open_dataset(output_path)
ds

## 4. Error Handling

**Objectives**: Verify how errors are handled: error messages and codes

### 4.1 Wrong URL

_**expected output**: 404 Client Error: Not Found for url_

In [ ]:
client = cads_api_client.ApiClient(f"{api_url}_1")
client.collections()

### 4.2 Missing collection

_**expected output**: 404 Client Error: Not Found for url_

In [ ]:
client = cads_api_client.ApiClient(api_url)
client.collection("missing_collection")

### 4.3 Unknown job

_**expected output**: 404 Client Error: Not Found for url_

In [ ]:
status_info = client.get_request("ffffffff-4455-6677-8899-aabbccddeeff")

### 4.4 Wrong request

**non blocking request using collection.submit**

_**expected output**: _collection.submit doesn't raise any Error_


In [ ]:
client = cads_api_client.ApiClient(api_url)
collection = client.collection("reanalysis-era5-pressure-levels")
remote = collection.submit( 
    target="output.grib",
    product_type="reanalysis", 
    variable="temperature", 
    pressure_level="1", 
    year="2222", 
    month="01", 
    day="25", 
    time="06:00",
    format="grib", 
)

_**expected output**: remote.wait_on_result raises a ProcessingFailedError_

In [ ]:
remote.wait_on_result()

_**expected output**: result status code is 400_

In [ ]:
results = remote.make_results()

In [ ]:
results.status_code

_**expected output**:_
- _result.json describes the error in the fields: "type", "title" and "detail"_
- _"detail" contains the error traceback_

In [ ]:
results.json

In [ ]:
headers = {"Content-Type"}